In [7]:
train_path="/home/luyy/projects/PTR/data/tacred/train.txt"
import json
file=open(train_path,'r')
lines=file.readlines()

In [2]:
from transformers import RobertaConfig, RobertaTokenizer, RobertaModel
model_classes = {'roberta': {'config': RobertaConfig,'tokenizer': RobertaTokenizer,'model':RobertaModel,}}
model_config = model_classes['roberta']
tokenizer = model_config['tokenizer'].from_pretrained('roberta-large')
special=[]
tokenizer.add_tokens(special)

2023-02-18 17:47:02.916918: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-18 17:47:03.139775: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-18 17:47:03.139803: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-18 17:47:03.999062: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

0

In [8]:
import torch
for line in lines:
  dic=json.loads(line)
  input_token = tokenizer.tokenize(dic['h']['name'])
  word_ids=tokenizer.convert_tokens_to_ids(input_token)
  print(torch.tensor(word_ids))
  print(dic)
  break


tensor([1301, 1073,  415])
{'token': ['Zagat', 'Survey', ',', 'the', 'guide', 'empire', 'that', 'started', 'as', 'a', 'hobby', 'for', 'Tim', 'and', 'Nina', 'Zagat', 'in', '1979', 'as', 'a', 'two-page', 'typed', 'list', 'of', 'New', 'York', 'restaurants', 'compiled', 'from', 'reviews', 'from', 'friends', ',', 'has', 'been', 'put', 'up', 'for', 'sale', ',', 'according', 'to', 'people', 'briefed', 'on', 'the', 'decision', '.'], 'h': {'name': 'Zagat', 'pos': [0, 1]}, 't': {'name': '1979', 'pos': [17, 18]}, 'relation': 'org:founded'}


In [9]:
model = model_config['model'].from_pretrained("roberta-large",return_dict=False)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
class_vec=0
for line in lines:
  dic=json.loads(line)
  if dic['relation']=='per:date_of_death':
    input_token = tokenizer.tokenize(dic['t']['name'])
    word_ids=tokenizer.convert_tokens_to_ids(input_token)
    embeddings = model.embeddings.word_embeddings(torch.tensor(word_ids))
    word_vec=torch.mean(embeddings,dim=0).unsqueeze(0)
    if isinstance(class_vec,int):
      class_vec=word_vec
    else:
      class_vec=torch.cat((class_vec,word_vec),0)
print(class_vec.size())


torch.Size([134, 1024])


In [11]:
class_vec1=0
for line in lines:
  dic=json.loads(line)
  if dic['relation']=='per:date_of_death':
    input_token = tokenizer.tokenize(dic['t']['name'])
    word_ids=tokenizer.convert_tokens_to_ids(input_token)
    embeddings = model.embeddings.word_embeddings(torch.tensor(word_ids))
    word_vec=torch.mean(embeddings,dim=0).unsqueeze(0)
    if isinstance(class_vec1,int):
      class_vec1=word_vec
    else:
      class_vec1=torch.cat((class_vec,word_vec),0)
print(class_vec1.size())

AttributeError: 'RobertaModel' object has no attribute 'most_similar'